# Menganalisis Risiko Gagal Bayar Peminjam

Tugas Anda adalah menyiapkan laporan untuk divisi kredit suatu bank. Anda akan mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman. Pihak bank sudah memiliki beberapa data mengenai kelayakan kredit nasabah.

Laporan Anda akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.

Tujuan proyek ini adalah mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman

## Buka *file* data dan baca informasi umumnya.

In [ ]:
# Muat semua *library*
import pandas as pd

# Muat datanya
credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Soal 1. Eksplorasi data

**Deskripsi Data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - pengidentifikasi untuk status perkawinan nasabah
- `family_status_id` - tanda pengenal status perkawinan
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah memiliki hutang pembayaran pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman

In [ ]:
# Mari kita lihat berapa banyak baris dan kolom yang dimiliki oleh dataset kita
credit_scoring.shape

(21525, 12)

In [ ]:
original_dataset_shape = credit_scoring.shape

In [ ]:
# Mari tampilkan N baris pertama
credit_scoring.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Terdapat beberapa masalah yang membutuhkan penyelidikan dan perubahan lebih lanjut:
- Beberapa nilai di kolom 'days_employed' merupakan nilai negatif.
- Kurangnya konsistensi dalam menggunakan huruf besar/kecil di kolom 'education'

In [ ]:
# Dapatkan informasi data
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
credit_scoring.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Total nilai untuk setiap kolom seharusnya 21525, namun pada kolom 'day_employed' dan 'toal_income' hanya 19351. Nilai yang hilang ada di kolom 'day_employed' dan total_income sebanyak 2174.

In [ ]:
# Mari kita lihat tabel yang difilter dengan nilai yang hilang di kolom pertama yang mengandung data yang hilang
credit_scoring[credit_scoring['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Nilai yang hilang tampak simetris pada kolom 'days_employed' dan 'total_income'

In [ ]:
# Mari kita terapkan beberapa kondisi untuk memfilter data dan melihat jumlah baris dalam tabel yang telah difilter.
missing_value = credit_scoring.loc[(credit_scoring['days_employed'].isna()) & (credit_scoring['total_income'].isna())]
missing_value


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Kesimpulan sementara**

1. Jumlah baris dalam tabel yang difilter sama dengan jumlah nilai yang hilang.
2. Langkah selanjutnya adalah menganalisa pengaruh data yang hilang terhadap keseluruhan data dengan cara :
- Menghitung persentase nilai yang hilang dibandingkan dengan keseluruhan data.
- Mencari nilai yang unik pada kolom-kolom lain yang berhubungan dengan data yang hilang.

In [ ]:
# Mari kita periksa nasabah yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang
a = (credit_scoring['days_employed'].isna().sum() / len(credit_scoring))*100
b = (credit_scoring['total_income'].isna().sum() / len(credit_scoring))*100
print("Persentase data yang hilang pada kolom 'day_employed' adalah {:.1f} %".format(round(a, 2)))
print("Persentase data yang hilang pada kolom 'total_income' adalah {:.1f} %".format(round(b, 2)))


Persentase data yang hilang pada kolom 'day_employed' adalah 10.1 %
Persentase data yang hilang pada kolom 'total_income' adalah 10.1 %


Data yang hilang untuk kedua kolom sekitar 10%, selanjutnya kita akan mencari tahu hubungan data yang hilang ini dengan nilai-nilai pada kolom-kolom lain.

In [ ]:
# Periksalah distribusinya
# Periksa nilai unik pada kolom-kolom yang mempunyai nilai hilang ('missing_value')
for column in missing_value:
    unique_value = sorted(missing_value[column].unique())
    
    print(f'{column} :')     
    print(f'{unique_value}\n') 

children :
[-1, 0, 1, 2, 3, 4, 5, 20]

days_employed :
[nan]

dob_years :
[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]

education :
["BACHELOR'S DEGREE", "Bachelor's Degree", 'PRIMARY EDUCATION', 'Primary Education', 'SECONDARY EDUCATION', 'SOME COLLEGE', 'Secondary Education', 'Some College', "bachelor's degree", 'primary education', 'secondary education', 'some college']

education_id :
[0, 1, 2, 3]

family_status :
['civil partnership', 'divorced', 'married', 'unmarried', 'widow / widower']

family_status_id :
[0, 1, 2, 3, 4]

gender :
['F', 'M']

income_type :
['business', 'civil servant', 'employee', 'entrepreneur', 'retiree']

debt :
[0, 1]

total_income :
[nan]

purpose :
['building a property', 'building a real estate', 'buy commercial real estate', 'buy real estate', 'buy residential real estate', 'bu

In [ ]:
#Fungsi untuk memeriksa distribusi pada tabel yang difilter 'missing_value'
def distribution_missing(missing_col):
    print(missing_value[missing_col].value_counts(normalize=True))

In [ ]:
# Distribusi untuk kolom 'children'
distribution_missing('children')

 0     0.661914
 1     0.218491
 2     0.093836
 3     0.016559
 20    0.004140
 4     0.003220
-1     0.001380
 5     0.000460
Name: children, dtype: float64


In [ ]:
# Distribusi untuk kolom 'days_employed'
distribution_missing('days_employed')

Series([], Name: days_employed, dtype: float64)


In [ ]:
# Distribusi untuk kolom 'dob_years'
distribution_missing('dob_years')

34    0.031739
40    0.030359
31    0.029899
42    0.029899
35    0.029439
36    0.028979
47    0.027139
41    0.027139
30    0.026679
28    0.026219
57    0.025759
58    0.025759
54    0.025299
38    0.024839
56    0.024839
37    0.024379
52    0.024379
39    0.023459
33    0.023459
50    0.023459
51    0.022999
45    0.022999
49    0.022999
29    0.022999
43    0.022999
46    0.022079
55    0.022079
48    0.021159
53    0.020239
44    0.020239
60    0.017939
61    0.017479
62    0.017479
64    0.017019
32    0.017019
27    0.016559
23    0.016559
26    0.016099
59    0.015639
63    0.013339
25    0.010580
24    0.009660
66    0.009200
65    0.009200
21    0.008280
22    0.007820
67    0.007360
0     0.004600
68    0.004140
69    0.002300
20    0.002300
71    0.002300
70    0.001380
72    0.000920
19    0.000460
73    0.000460
Name: dob_years, dtype: float64


In [ ]:
#Distribusi untuk kolom 'education'
distribution_missing('education')

secondary education    0.647654
bachelor's degree      0.228151
SECONDARY EDUCATION    0.030819
Secondary Education    0.029899
some college           0.025299
Bachelor's Degree      0.011500
BACHELOR'S DEGREE      0.010580
primary education      0.008740
Some College           0.003220
SOME COLLEGE           0.003220
Primary Education      0.000460
PRIMARY EDUCATION      0.000460
Name: education, dtype: float64


In [ ]:
#Distribusi untuk kolom 'education_id'
distribution_missing('education_id')

1    0.708372
0    0.250230
2    0.031739
3    0.009660
Name: education_id, dtype: float64


In [ ]:
#Distribusi untuk kolom 'family_status'
distribution_missing('family_status')

married              0.568997
civil partnership    0.203312
unmarried            0.132475
divorced             0.051518
widow / widower      0.043698
Name: family_status, dtype: float64


In [ ]:
#Distribusi untuk kolom 'family_status_id'
distribution_missing('family_status_id')

0    0.568997
1    0.203312
4    0.132475
3    0.051518
2    0.043698
Name: family_status_id, dtype: float64


In [ ]:
#Distribusi untuk kolom 'gender'
distribution_missing('gender')

F    0.682613
M    0.317387
Name: gender, dtype: float64


In [ ]:
#Distribusi untuk kolom 'income_type'
distribution_missing('income_type')

employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64


In [ ]:
#Distribusi untuk kolom 'total_income'
distribution_missing('total_income')

Series([], Name: total_income, dtype: float64)


In [ ]:
#Distribusi untuk kolom 'purpose'
distribution_missing('purpose')

having a wedding                            0.042318
to have a wedding                           0.037259
wedding ceremony                            0.034959
construction of own property                0.034499
housing transactions                        0.034039
buy real estate                             0.033119
transactions with my real estate            0.032659
purchase of the house for my family         0.032659
housing renovation                          0.032199
transactions with commercial real estate    0.032199
buy commercial real estate                  0.030819
buying property for renting out             0.029899
property                                    0.028519
buy residential real estate                 0.028059
real estate transactions                    0.028059
housing                                     0.027599
building a property                         0.027139
cars                                        0.026219
going to university                         0.

In [ ]:
#Distribusi untuk kolom 'debt'
distribution_missing('debt')

0    0.921803
1    0.078197
Name: debt, dtype: float64


Berikut ini temuan untuk kolom-kolom yang berkaitan dengan data yang hilang selain kolom 'days_employed ' dan 'total_income'.
1. 'children', terdapat nilai negatif
2. 'education', beberapa tingkat pendidikan yang sama dituliskan secara berbeda
3. 'education_id', tidak ada yang aneh
4. 'family_status', tidak ada yang aneh
5. 'family_status_id ', tidak ada yang aneh
6. 'gender', tidak ada yang aneh
7. 'income_type', tidak ada yang aneh
8. 'debt', tidak ada yang aneh
9. 'purpose', beberapa tujuan yang sama dituliskan secara berbeda



**Kemungkinan penyebab hilangnya nilai dalam data**

Nilai yang hilang terlihat memiliki pola, nilai yang hilang pada kolom 'days_employed' dan 'total_income' ada dibaris yang sama. Kemungkinan calon nasabah memilih untuk tidak mengisi data pekerjaan untuk lama bekerja dan data pendapatan karena bersifat pribadi.

Berikutnya kita memeriksa keseluruhan dataset dan membandingkannya dengan data yang telah difilter untuk pengambilan kesimpulan

In [ ]:
# Memeriksa distribusi di seluruh *dataset*
column_check = ['children','education','family_status','gender','income_type','debt','purpose']

def check_dataset(data):
    for column in column_check:
        print(f'{column} :')
        print(f'{credit_scoring[column].unique()}\n')
check_dataset(column_check)

children :
[ 1  0  3  2 -1  4 20  5]

education :
["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']

family_status :
['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']

gender :
['F' 'M' 'XNA']

income_type :
['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']

debt :
[0 1]

purpose :
['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transaction

In [ ]:
# Distribusi untuk keselurhan dataset
def distribution_dataset(dataset_col):
    print(credit_scoring[dataset_col].value_counts(normalize=True))

In [ ]:
# Distribusi untuk kolom 'children'
distribution_dataset('children')

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64


In [ ]:
# Distribusi untuk kolom 'days_employed'
distribution_dataset('days_employed')

-327.685916     0.000052
-1580.622577    0.000052
-4122.460569    0.000052
-2828.237691    0.000052
-2636.090517    0.000052
                  ...   
-7120.517564    0.000052
-2146.884040    0.000052
-881.454684     0.000052
-794.666350     0.000052
-3382.113891    0.000052
Name: days_employed, Length: 19351, dtype: float64


In [ ]:
# Distribusi untuk kolom 'dob_years'
distribution_dataset('dob_years')

35    0.028664
40    0.028293
41    0.028200
34    0.028014
38    0.027782
42    0.027735
33    0.026992
39    0.026620
31    0.026016
36    0.025784
44    0.025412
29    0.025319
30    0.025087
48    0.024994
37    0.024948
50    0.023879
43    0.023833
32    0.023693
49    0.023600
28    0.023368
45    0.023089
27    0.022904
56    0.022625
52    0.022485
47    0.022300
54    0.022253
46    0.022067
58    0.021417
57    0.021370
53    0.021324
51    0.020813
59    0.020627
55    0.020581
26    0.018955
60    0.017515
25    0.016585
61    0.016492
62    0.016353
63    0.012497
64    0.012311
24    0.012265
23    0.011800
65    0.009013
66    0.008502
22    0.008502
67    0.007758
21    0.005157
0     0.004692
68    0.004599
69    0.003949
70    0.003020
71    0.002695
20    0.002369
72    0.001533
19    0.000650
73    0.000372
74    0.000279
75    0.000046
Name: dob_years, dtype: float64


In [ ]:
# Distribusi untuk kolom 'education'
distribution_dataset('education')

secondary education    0.638792
bachelor's degree      0.219187
SECONDARY EDUCATION    0.035865
Secondary Education    0.033031
some college           0.031034
BACHELOR'S DEGREE      0.012729
Bachelor's Degree      0.012451
primary education      0.011614
Some College           0.002184
SOME COLLEGE           0.001347
PRIMARY EDUCATION      0.000790
Primary Education      0.000697
graduate degree        0.000186
GRADUATE DEGREE        0.000046
Graduate Degree        0.000046
Name: education, dtype: float64


In [ ]:
# Distribusi untuk kolom 'education_id'
distribution_dataset('education_id')

1    0.707689
0    0.244367
2    0.034564
3    0.013101
4    0.000279
Name: education_id, dtype: float64


In [ ]:
# Distribusi untuk kolom 'family_status'
distribution_dataset('family_status')

married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64


In [ ]:
# Distribusi untuk kolom 'family_status_id'
distribution_dataset('family_status_id')

0    0.575145
1    0.194053
4    0.130685
3    0.055517
2    0.044599
Name: family_status_id, dtype: float64


In [ ]:
# Distribusi untuk kolom 'gender'
distribution_dataset('gender')

F      0.661370
M      0.338583
XNA    0.000046
Name: gender, dtype: float64


In [ ]:
# Distribusi untuk kolom 'income_type'
distribution_dataset('income_type')

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
paternity / maternity leave    0.000046
student                        0.000046
Name: income_type, dtype: float64


In [ ]:
# Distribusi untuk kolom 'debt'
distribution_dataset('debt')

0    0.919117
1    0.080883
Name: debt, dtype: float64


In [ ]:
# Distribusi untuk kolom 'total_income'
distribution_dataset('total_income')

42413.096    0.000103
17312.717    0.000103
31791.384    0.000103
14427.878    0.000052
20837.034    0.000052
               ...   
27715.458    0.000052
23834.534    0.000052
26124.613    0.000052
28692.182    0.000052
41428.916    0.000052
Name: total_income, Length: 19348, dtype: float64


In [ ]:
# Distribusi untuk kolom 'purpose'
distribution_dataset('purpose')

wedding ceremony                            0.037027
having a wedding                            0.036098
to have a wedding                           0.035958
real estate transactions                    0.031405
buy commercial real estate                  0.030848
housing transactions                        0.030337
buying property for renting out             0.030337
transactions with commercial real estate    0.030244
purchase of the house                       0.030058
housing                                     0.030058
purchase of the house for my family         0.029779
construction of own property                0.029501
property                                    0.029454
transactions with my real estate            0.029268
building a real estate                      0.029082
buy real estate                             0.028990
building a property                         0.028804
purchase of my own house                    0.028804
housing renovation                          0.

Dari hasil pemeriksaan distrisbusi, terlihat nilai antara keseluruhan dataset **credit_scoring** dan tabel yang difilter **missing_values** memiliki pola yang sama.

**Kesimpulan sementara**

Kode diatas memeriksa distribusi untuk keseluruhan dataset untuk semua kolom kategorik dan beberapa kolom numerik. 

Beberapa kolom numerik diabaikan/tidak diperiksa karena :
- Dapat diwakilkan oleh kolom kategorik yang berkaitan
- Data yang sangat banyak di kolom numerik sehingga sulit dianalisa secara visual

Distribusi dataset yang asli dan yang telah difilter relatif sama untuk kolom berikut:
- 'children'
- 'education'
- 'family_status'
- 'debt'
- 'purpose'

Ada sedikit perbedaan pada kolom berikut ini:
- 'gender', terdapat value 'XNA' di dataset asli yang tidak ditemukan di tabel yang difilter.
- 'income_type', terdapat value 'paternity / maternity leave' di dataset asli yang tidak ditemukan di tabel yang difilter.

**Kesimpulan**

Secara umum ada kesamaan data antara tabel yang difilter dan dataset kecuali untuk kolom 'gender' dan 'purpose'.

Berikut ini kesimpulan yang bisa ambil sampai saat ini:
- Tidak ada perbedaan yang signifikan antara tabel yang difilter dengan dataset secara keseluruhan.
- Data yang hilang memiliki pola untuk setiap kolom 'days_employed' dan 'total_income'.
- Baris yang hilang sekitar 10% dari total dataset, dengan asumsi calon nasabah tidak mengisi datanya.




Langkah berikutnya adalah transformasi data dengan mengatasi masalah duplikat dengan memeriksa ejaan, memperbaiki pencatatan, dll

## Transformasi data

Pengecekan data di kolom 'education'

In [ ]:
# Mari kita lihat semua nilai di kolom pendidikan untuk memeriksa ejaan apa yang perlu diperbaiki
credit_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [ ]:
# Fungsi mengubah ke huruf kecil
def huruf_kecil(data):
    return data.lower()

In [ ]:
# Perbaiki pencatatan jika diperlukan
credit_scoring['education'] = credit_scoring['education'].apply(huruf_kecil)

In [ ]:
# Periksa semua nilai di kolom untuk memastikan bahwa kita telah memperbaikinya dengan tepat
credit_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Pengecekan data di kolom 'children'

In [ ]:
# Mari kita lihat distribusi nilai pada kolom `children`
credit_scoring['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Terdapat value 2 value yang tidak masuk diakal yaitu -1 dan 20.
- -1 akan diganti menjadi 1 karena kemungkinan ada kesalahan input tanda '-'
- 20 akan diganti menjadi 2 karena kemungkinan ada kesalahan penambahan digit '0'

In [ ]:
# [perbaiki data berdasarkan keputusan Anda]
credit_scoring['children'] = credit_scoring['children'].replace(-1,1)
credit_scoring['children'] = credit_scoring['children'].replace(20,2)

In [ ]:
# Periksa kembali kolom `children` untuk memastikan bahwa semuanya telah diperbaiki
credit_scoring['children'].unique()

array([1, 0, 3, 2, 4, 5])

Pemeriksaan data di kolom 'days_employed'

In [ ]:
# Temukan data yang bermasalah di `days_employed` 
print(credit_scoring['days_employed'])

# Mencari nilai negatif di kolom 'days_employed' dan menghitung persentasenya
credit_scoring[credit_scoring['days_employed'] < 0]['days_employed'].count() / len(credit_scoring)

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64


0.7389547038327526

In [ ]:
# Terdapat sekitar 82% nilai negatif pada kolom 'days_employed'
# Menghilangkan nilai negatif pada kolom 'days_employed'
credit_scoring['days_employed'] = abs(credit_scoring['days_employed'])

In [ ]:
# Pemeriksaan nilai negatif pada kolom 'days_employed'
credit_scoring[credit_scoring['days_employed'] < 0]['days_employed'].count() / len(credit_scoring)

0.0

In [ ]:
# Mencari lama kerja diatas 60 tahun di kolom 'days_employed'
credit_scoring.loc[credit_scoring['days_employed'] > (60 * 365),'days_employed'].count()

3445

**Kesimpulan Sementara**

1. Terdapat 3445 lama kerja diatas 60 tahun. Data ini kurang valid, jika pada umumnya seseorang mulai bekerja pada usia 20 tahun ditambah masa kerja 60 tahun, maka usia total sebesar 80 tahun atau sudah memasuki usia pensiun/tidak bekerja. 
2. Dengan demikian nilai ini sebaiknya diganti sesuai dengan usia calon nasabah di kolom 'dob_years' dengan asumsi usia awal kerja 20 tahun.

In [ ]:
# Mengganti usia calon nasabah yang masa kerjanya diatas 60 tahun menjadi 50 tahun
credit_scoring.loc[credit_scoring['days_employed'] > (60 * 365),'days_employed'] = (credit_scoring['dob_years'] - 20) * 365

In [ ]:
# Pemeriksaan ulang jumlah calon nasabah yang masa kerjanya diatas 60 tahun
credit_scoring.loc[credit_scoring['days_employed'] > (60 * 365),'days_employed'].count()

0

In [ ]:
# Periksa `dob_years` untuk nilai yang mencurigakan dan hitung persentasenya
sorted(credit_scoring['dob_years'].unique())

[0,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

In [ ]:
# Terdapat usia 0 di kolom 'dob_years'
# Hitung jumlah usia 0 di kolom 'dob_years'
credit_scoring.loc[credit_scoring['dob_years'] == 0,'dob_years'].count()

101

In [ ]:
# Persentase usia calon nasabah yang tidak valid
(credit_scoring.loc[credit_scoring['dob_years'] == 0,'dob_years'].count() / len(credit_scoring))*100

0.4692218350754936

Persentase usia calon nasabah yang bernilai 0 relatif kecil sekitar 0.47%
Karena usia 0 tidak mungkin untuk melakukan peminjaman, maka diganti dengan nilai terkecil yang valid yakni 19

In [ ]:
#Mengganti usia calon nasabah yang bernilai 0 menjadi 19
credit_scoring['dob_years'] = credit_scoring['dob_years'].replace(0,19)

In [ ]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
credit_scoring.loc[credit_scoring['dob_years'] == 0,'dob_years'].count()

0

Pemeriksaan kolom `family_status`

In [ ]:
# Mari kita lihat nilai untuk kolom ini
credit_scoring['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

Tidak ada masalah di kolom 'family_status

**Pemeriksaan kolom `gender`**

In [ ]:
# Mari kita liat nilai dalam kolom ini
credit_scoring['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [ ]:
# Terdapat value 'XNA' yang tidak dapat diidentifikasi
# Mencari jumlah baris yang memiliki value 'XNA'
credit_scoring.loc[credit_scoring['gender'] == 'XNA', 'gender'].count()

1

In [ ]:
# Hanya ada 1 baris yang memiliki value 'XNA' pada kolom 'gender'
# Kemungkinan calon nasabah tidak memberikan data jenis kelaminnya
# Dengan demikian baris ini dapat dihilangkan

# Menghilangkan baris dengan value 'XNA' pada kolom 'gender'
credit_scoring = credit_scoring.loc[credit_scoring['gender'] != 'XNA']

In [ ]:
# Pemeriksaan ulang pada kolom 'gender'
credit_scoring.loc[credit_scoring['gender'] == 'XNA', 'gender'].count()

0

Pemeriksaan kolom `income_type`

In [ ]:
# Mari kita lihat nilai dalam kolom ini
credit_scoring['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

Tidak ada masalah untuk kolom 'income_type'

**Pemeriksaan Duplikat**

In [ ]:
# Periksa duplikat
credit_scoring.duplicated().sum()

71

In [ ]:
# Menghapus duplikat
credit_scoring.drop_duplicates(inplace = True)

In [ ]:
# Periksa ulang duplikat
credit_scoring.duplicated().sum()

0

In [ ]:
# Periksa ukuran dataset yang sekarang Anda miliki setelah manipulasi pertama yang Anda lakukan
new_dataset_shape = credit_scoring.shape
print(new_dataset_shape)
print(original_dataset_shape)

# Menghitung perubahan baris dataset
print(original_dataset_shape[0] - new_dataset_shape[0])

# Menghitung persentase perubahan dataset
print(((original_dataset_shape[0] - new_dataset_shape[0])/original_dataset_shape[0])*100 )

(21453, 12)
(21525, 12)
72
0.3344947735191638


Dataset baru : 21453 baris dan 12 kolom
Dataset asli : 21525 baris dan 12 kolom

Ada 72 baris yang dihapus yang merupakan baris dari :
- 71 data duplikat
- 1 data jenis kelamin 'XNA' yang tidak dapat diidentifikasi dari kolom 'gender'

Persentase data yang dihapus sekitar 0.3% dari dataset asli.


# Bekerja dengan nilai yang hilang

In [ ]:
# Urutan usia calon nasabah dari kolom 'dob_years'
sorted(credit_scoring['dob_years'].unique())

[19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

### Memperbaiki nilai yang hilang di `total_income`

Kategori Usia

In [ ]:
# Mari kita tulis sebuah fungsi untuk menghitung kategori usia
def age_category(usia):
    if usia < 30:
        category = 'Dewasa Muda'
    elif 29 < usia < 40:
        category = 'Dewasa'
    elif 39 < usia < 50:
        category = 'Paruh Baya'
    elif 49 < usia < 60:
        category = 'Lansia'
    else:
        category = 'Tua'
    return category    

In [ ]:
# Lakukan pengujian untuk melihat apakah fungsi Anda bekerja atau tidak
age_category(17)

'Dewasa Muda'

In [ ]:
# Buatlah kolom baru berdasarkan fungsi
credit_scoring['age_category'] = credit_scoring['dob_years'].apply(age_category)
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Paruh Baya
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Dewasa
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,Dewasa
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Dewasa
4,0,12045.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Lansia


In [ ]:
# Periksa bagaimana nilai di dalam kolom baru
credit_scoring['age_category'].value_counts()

Dewasa         5662
Paruh Baya     5354
Lansia         4657
Dewasa Muda    3280
Tua            2500
Name: age_category, dtype: int64

In [ ]:
# Buat tabel tanpa nilai yang hilang dan tampilkan beberapa barisnya untuk memastikan semuanya berjalan dengan baik
data_clean = credit_scoring.loc[~(credit_scoring['days_employed'].isna()) & ~(credit_scoring['total_income'].isna())]

In [ ]:
# Perhatikan nilai rata-rata untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
data_clean['total_income'].mean()

26787.26668764858

In [ ]:
# Perhatikan nilai median untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
data_clean['total_income'].median()


23201.8735

In [ ]:
# Memeriksa nilai gaji terkecil dan terbesar di kolom 'total_income'
print(credit_scoring['total_income'].min())
print(credit_scoring['total_income'].max())

3306.762
362496.645


- Nilai rata-rata untuk total_income  = 26787.26668764858
- Nilai tengah untuk total_income = 23201.8735
- Nilai terkecil untuk total_income = 3306.762
- Nilai terbesar untuk total_income = 362496.645

Dari data diatas ditemukan outlier yang signifikan, sehingga lebih baik menggunakan median untuk mengisi nilai yang hilang

In [ ]:
# Memasang variable yang akan menampung nilai unik dari kolom 'income_type'
income_values = credit_scoring['income_type'].unique()

# Dengan nilai unik pada kolom 'income_type', kita bisa melakukan looping
# untuk mendapatkan median pada setiap nilai untik tersebut
# setelah itu kita dapat mengisi nilai yang hilang dengan median untuk setiap kategori 'income_type'
for value in income_values:
    median = data_clean.loc[data_clean['income_type'] == value, 'total_income'].median()
    credit_scoring.loc[(credit_scoring['income_type'] == value) & (credit_scoring['total_income'].isna()), 'total_income'] = median

In [ ]:
# Periksa jumlah entri di kolom 'total_income'
credit_scoring['total_income'].isna().sum()

0

In [ ]:
# Periksa jumlah entri di dataset
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


Data untuk kolom 'total_income' sudah sesuai dengan kolom-kolom lainnya kecuali data pada kolom 'days_employed.

In [ ]:
data_clean.groupby(['income_type'])['total_income'].mean()

income_type
business                       32386.741818
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21940.394503
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64

In [ ]:
data_clean.groupby(['income_type'])['total_income'].median()

income_type
business                       27571.0825
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

###  Memperbaiki nilai di `days_employed`

In [ ]:
# Distribusi median dari `days_employed` berdasarkan parameter yang Anda identifikasi
for column in ['age_category', 'gender', 'family_status', 'education', 'income_type', 'total_income' ]:
    print(credit_scoring.groupby(column)['days_employed'].median())
    print()

age_category
Dewasa          1601.784231
Dewasa Muda     1000.328028
Lansia          4796.767897
Paruh Baya      2111.489906
Tua            15330.000000
Name: days_employed, dtype: float64

gender
F    2531.439644
M    1661.491868
Name: days_employed, dtype: float64

family_status
civil partnership     1943.995523
divorced              2398.424010
married               2300.117387
unmarried             1456.183548
widow / widower      12775.000000
Name: days_employed, dtype: float64

education
bachelor's degree      1893.079012
graduate degree        5660.057032
primary education      3043.933615
secondary education    2385.737683
some college           1206.479128
Name: days_employed, dtype: float64

income_type
business                        1546.333214
civil servant                   2689.368353
employee                        1574.202821
entrepreneur                     520.848083
paternity / maternity leave     3296.759962
retiree                        14600.000000
student      

In [ ]:
# Distribusi rata-rata dari `days_employed` berdasarkan parameter yang Anda identifikasi
for column in ['age_category', 'gender', 'family_status', 'education', 'income_type', 'total_income' ]:
    print(credit_scoring.groupby(column)['days_employed'].mean())
    print()

age_category
Dewasa          2049.019141
Dewasa Muda     1187.971698
Lansia          6762.690337
Paruh Baya      2910.403876
Tua            13435.192297
Name: days_employed, dtype: float64

gender
F    5112.006805
M    3248.397551
Name: days_employed, dtype: float64

family_status
civil partnership    4093.130368
divorced             4575.167339
married              4443.260269
unmarried            3369.004341
widow / widower      9670.055089
Name: days_employed, dtype: float64

education
bachelor's degree      3585.088950
graduate degree        7882.624358
primary education      6613.689872
secondary education    4852.103366
some college           2226.466075
Name: days_employed, dtype: float64

income_type
business                        2111.470404
civil servant                   3399.896902
employee                        2326.499216
entrepreneur                     520.848083
paternity / maternity leave     3296.759962
retiree                        14285.037758
student           

Karena data memiliki outlier yang significant pada beberapa kolom diatas, maka median sebaiknya digunakan untuk mengisi nilai yang hilang

In [ ]:
# Memasang variable yang akan menampung nilai unik dari kolom 'income_type'
income_values = credit_scoring['income_type'].unique()

# Dengan nilai unik pada kolom 'income_type', kita bisa melakukan looping
# untuk mendapatkan median pada setiap nilai untik tersebut
# setelah itu kita dapat mengisi nilai yang hilang dengan median untuk setiap kategori 'income_type'
for value in income_values:
    median = data_clean.loc[data_clean['income_type'] == value, 'days_employed'].median()
    credit_scoring.loc[(credit_scoring['income_type'] == value) & (credit_scoring['days_employed'].isna()), 'days_employed'] = median

In [ ]:
# Periksa jumlah entri di kolom 'days_employed'
credit_scoring['days_employed'].isna().sum()

0

In [ ]:
# Periksa entri di semua kolom - pastikan kita memperbaiki semua nilai yang hilang
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


Semua nilai yang hilang telah diperbaiki dengan jumlah data setiap kolom sebanyak 21453 

## Pengkategorian Data

Pengkategorian kolom kategorik dan numerik untuk pengambilan keputusan

In [ ]:
# Tampilkan nilai data yang Anda pilih untuk pengkategorian
column_category = ['gender','children','education','age_category']

Memeriksa Nilai Unik

In [ ]:
# Periksa nilai unik
for x in column_category:
    print(x)
    print(credit_scoring[x].unique())
    print()

gender
['F' 'M']

children
[1 0 3 2 4 5]

education
["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']

age_category
['Paruh Baya' 'Dewasa' 'Lansia' 'Dewasa Muda' 'Tua']



Mengkategorkan data berdasarkan ada atau tidaknya pinjaman calon nasabah


In [ ]:
# Mari kita tulis sebuah fungsi untuk mengategorikan data berdasarkan topik umum
def rename_debt(value):
    if value == 1:
        desc = 'Memiliki Hutang'
    else:
        desc = 'Tidak Punya Hutang'
    return desc

In [ ]:
# Buat kolom yang memuat kategori
credit_scoring['debt_description'] = credit_scoring['debt'].apply(rename_debt)

credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,debt_description
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Paruh Baya,Tidak Punya Hutang
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Dewasa,Tidak Punya Hutang
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,Dewasa,Tidak Punya Hutang
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Dewasa,Tidak Punya Hutang
4,0,12045.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Lansia,Tidak Punya Hutang


In [ ]:
# Mari kita tulis sebuah fungsi untuk mengategorikan tujuan
def purpose_cat(value):
    if value == 'building a property':
        desc = 'property'
    elif value == 'building a real estate':
        desc = 'property'
    elif value == 'buy commercial real estate':
        desc = 'property'
    elif value == 'buy real estate':
        desc = 'property'
    elif value == 'buy residential real estate':
        desc = 'property'      
    elif value == 'buying property for renting out':
        desc = 'property'
    elif value == 'construction of own property':
        desc = 'property'
    elif value == 'housing':
        desc = 'property'
    elif value == 'housing renovation':
        desc = 'property'
    elif value == 'housing transactions':
        desc = 'property'
    elif value == 'purchase of my own house':
        desc = 'property'        
    elif value == 'purchase of the house':
        desc = 'property'        
    elif value == 'purchase of the house for my family':
        desc = 'property'
    elif value == 'real estate transactions':
        desc = 'property'
    elif value == 'transactions with commercial real estate':
        desc = 'property'
    elif value == 'transactions with my real estate':
        desc = 'property'
    elif value == 'property':
        desc = 'property'
    elif value == 'buying a second-hand car':
        desc = 'car'      
    elif value == 'buying my own car':
        desc = 'car'  
    elif value == 'car':
        desc = 'car'  
    elif value == 'car purchase':
        desc = 'car'      
    elif value == 'cars':
        desc = 'car'  
    elif value == 'purchase of a car':
        desc = 'car'  
    elif value == 'second-hand car purchase':
        desc = 'car'      
    elif value == 'to buy a car':
        desc = 'car'  
    elif value == 'to own a car':
        desc = 'car'  
    elif value == 'education':
        desc = 'education' 
    elif value == 'getting an education':
        desc = 'education'
    elif value == 'getting higher education':
        desc = 'education'
    elif value == 'going to university':
        desc = 'education' 
    elif value == 'profile education':
        desc = 'education'
    elif value == 'supplementary education':
        desc = 'education'
    elif value == 'to become educated':
        desc = 'education' 
    elif value == 'to get a supplementary education':
        desc = 'education'
    elif value == 'university education':
        desc = 'education'
    else:
        desc = 'wedding'
    return desc

In [ ]:
# Buat kolom yang memuat kategori
credit_scoring['purpose_category'] = credit_scoring['purpose'].apply(purpose_cat)

credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,debt_description,purpose_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Paruh Baya,Tidak Punya Hutang,property
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Dewasa,Tidak Punya Hutang,car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,Dewasa,Tidak Punya Hutang,property
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Dewasa,Tidak Punya Hutang,education
4,0,12045.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Lansia,Tidak Punya Hutang,wedding


## Memeriksa hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan melakukan pelunasan tepat waktu?**

In [ ]:
# Periksa data anak dan data pelunasan tepat waktu
print(credit_scoring[['children','debt']])

# Hitung gagal bayar berdasarkan jumlah anak
credit_scoring.groupby('children')['debt'].mean()


       children  debt
0             1     0
1             1     0
2             0     0
3             3     0
4             0     0
...         ...   ...
21520         1     0
21521         0     0
21522         1     1
21523         3     1
21524         2     0

[21453 rows x 2 columns]


children
0    0.075444
1    0.091658
2    0.094925
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64

**Kesimpulan**
Hubungan memiliki banyak anak terhadap gagal bayar hutang tidak kuat.


**Apakah terdapat korelasi antara status keluarga dengan pelunasan tepat waktu?**

In [ ]:
# Periksa data status keluarga dan pelunasan tepat waktu
print(credit_scoring[['family_status','debt']])

# Hitung gagal bayar berdasarkan status keluarga
credit_scoring.groupby('family_status')['debt'].mean()


           family_status  debt
0                married     0
1                married     0
2                married     0
3                married     0
4      civil partnership     0
...                  ...   ...
21520  civil partnership     0
21521            married     0
21522  civil partnership     1
21523            married     1
21524            married     0

[21453 rows x 2 columns]


family_status
civil partnership    0.093494
divorced             0.071130
married              0.075452
unmarried            0.097509
widow / widower      0.065693
Name: debt, dtype: float64

**Kesimpulan**

Hubungan status keluarga dengan gagal bayar hutang tidak kuat. Namun janda(widow/widower) memiliki kemampuan bayar yang paling baik dan yang terburuk adalah belum menikah (unmarried).


**Apakah terdapat korelasi antara tingkat pendapatan dengan membayar kembali tepat waktu?**

In [ ]:
# Fungsi pengkategorian pendapatan
def income_category(income):
    if income < 4500:
        category = 'Lower'
    elif 4500 <= income < 13000:
        category = 'Middle'
    else:
        category = 'Upper'
    return category

In [ ]:
# Buat kolom yang memuat kategori
credit_scoring['income_category'] = credit_scoring['total_income'].apply(income_category)
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,debt_description,purpose_category,income_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Paruh Baya,Tidak Punya Hutang,property,Upper
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Dewasa,Tidak Punya Hutang,car,Upper
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,Dewasa,Tidak Punya Hutang,property,Upper
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Dewasa,Tidak Punya Hutang,education,Upper
4,0,12045.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Lansia,Tidak Punya Hutang,wedding,Upper


In [ ]:
# Periksa data tingkat pendapatan dan pelunasan tepat waktu
print(credit_scoring[['income_category','debt']])

# Hitung gagal bayar berdasarkan tingkat pendapatan
credit_scoring.groupby('income_category')['debt'].mean()


      income_category  debt
0               Upper     0
1               Upper     0
2               Upper     0
3               Upper     0
4               Upper     0
...               ...   ...
21520           Upper     0
21521           Upper     0
21522           Upper     1
21523           Upper     1
21524           Upper     0

[21453 rows x 2 columns]


income_category
Lower     0.062500
Middle    0.075188
Upper     0.081920
Name: debt, dtype: float64

**Kesimpulan**
relasi terbalik antara besar penghasilan dengan gagal bayar, namun relasi ini relatif lemah.
Semakin besar pendapatan maka kemungkinan gagal bayar lebh tinggi.

**Bagaimana tujuan kredit memengaruhi tingkat gagal bayar?**

In [ ]:
# Periksa persentase tingkat gagal bayar untuk setiap tujuan kredit dan lakukan penganalisisan
credit_scoring.groupby('purpose_category')['debt'].mean()

purpose_category
car          0.093590
education    0.092200
property     0.072340
wedding      0.080034
Name: debt, dtype: float64

**Kesimpulan**
- Relasi antara tujuan dengan gagal bayar lemah
- Tujuan digroup 'car' memiliki kecenderungan gagal bayar yang paling tinggi
- Tujuan digroup 'property' memiliki kecenderungan gagal bayar yang paling rendah

# Kesimpulan umum 

Dari hasil analisis resiko gagal bayar peminjam tidak ditemukan hubungan yang kuat secara umum. Ditemukan hubungan yang lemah dengan kondisi mempunyai anak lebih banyak, pendapatan dan tujuan peminjaman terhadap kecenderungan gagal bayar. Prediksi akan lebih baik jika kita menggabungkan beberapa analisis. Kita dapat menyimpulkan gambaran peminjam yang baik adalah yang tidak mempunyai anak dengan tujuan peminjaman perumahan.
